# OPEA ChatQnA Blueprint with Milvus as a Vector Database Deployment (DOCKER)

The following notebook will instruct you to deploy a RAG chatbot based on OPEA blueprint ChatQnA blueprints using Docker containers

## Intro : The Open Platform for Enterprise (OPEA) Project 
OPEA uses microservices to create high-quality GenAI applications for enterprises, simplifying the scaling and deployment process for production. These microservices leverage a service composer that assembles them into a megaservice thereby creating real-world enterprise AI applications.

It’s important to familiarize yourself with the MAIN key elements of OPEA:

1. [**GenAIComps**](https://github.com/opea-project/GenAIComps) 
A collection of microservice components that form a service-based toolkit. Each microservice is designed to perform a specific function or task within the GenAI application architecture. By breaking down the system into these smaller, self-contained services, microservices promote modularity, flexibility, and scalability. This modular approach allows developers to independently develop, deploy, and scale individual components of the application, making it easier to maintain and evolve over time. All of the microservices are containerized, allowing cloud native deployment. Here, you will find contributions to multiple partners/communities to further construction.

2. [**GenAIExamples**](https://github.com/opea-project/GenAIExamples)
While *GenAIComps* offers a range of microservices, *GenAIExamples* provides practical, deployable solutions to help users implement these services effectively. This repo provides use-case-based applications that demonstrate how the OPEA architecture can be leveraged to build and deploy real-world GenAI applications. In the repo, developers can find practical resources such as Docker Compose files and Kubernetes Helm charts, which help streamline the deployment and scaling of these applications. These resources allow users to quickly set up and run the examples in local or cloud environments, ensuring a seamless experience.

# 1. Prepare the environment

This notebook uses Docker as an engine, you can deploy this example on any of the available Cloud providers following [THIS]([here](https://opea-project.github.io/latest/getting-started/README.html)) guide . 

Once your plaftorm is provisioned, follow the below steps to run the example:.

## 1.1 Install Docker Engine (Linux)

This is the only requirement to have in your environment since OPEA is based on a microservices architecture.

Use the command below to install docker:

In [ ]:
!curl -fsSL https://get.docker.com -o get-docker.sh
!sudo sh get-docker.sh --version 26.1
!sudo gpasswd -a devcloud docker

## 1.2 Clone GenAIExamples Repo

As mentioned, end-to-end blueprints are provided on OPEA [GenAIExamples repo](link), you can see there other examples available like : AgentsQnA, AudioQnA and MultimodalQnA amoung others.

Clone the repo:

In [ ]:
!git clone https://github.com/opea-project/GenAIExamples.git

# 2. Deployment

Let's now explore the OPEA ChatQnA RAG deployment. As mentioned, it's a microservices blueprint designed for scalability, resilience, and flexibility. In this task you will explore each microservice, the purpose of exploring each microservice is to help you understand how each component contributes to the overall application. This learning path will guide you through the system, illustrating the role of each service and how they work together.

Each service can scale individually based on demand, optimizing resources and performance. Additionally, microservices improve fault isolation—if one service fails, it doesn’t disrupt the entire system. This architecture supports efficient maintenance, rapid updates, and adaptability, making it ideal for responding to changing business needs and user demands.

Every OPEA configuration is built on three main parts:

![microservices-arch](./Images/microservices-arch.png)

- **Megaservice** : Microservice "orchestrator". When deploying an end-to-end application with multiple parts involved, there is needed to specify how the flow will be within the microservices. You can learn more from [OPEA documentation](https://github.com/opea-project/GenAIComps?tab=readme-ov-file#megaservice)

- **Gateway** : A gateway is the interface for users to access to the `megaservice` It acts as the entry point for incoming requests, routing them to the appropriate Microservices within the megaservice architecture.

- **Microservice** : Each individual microservice part of the end-to-end application like : **embeddings**, **retrievers**, **LLM** and **vector databases** among others.

You can see below the architecture you will be deploying `ChatQnA`

!["opea-rag"](./Images/opea_rag.png)

## 2.1 Set your enviroment variables 

Each microservice has a set of configurable variables that will specifcy the containter behaviour.For this example, we can specify which Models will be used by the `Embedding`, `LLM` and `reranker`.

In [ ]:
import os
import requests
import json

# Set environment variables
os.environ["host_ip"] = os.popen("hostname -I | awk '{print $1}'").read().strip()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_huggingface_key"
os.environ["EMBEDDING_MODEL_ID"] = "BAAI/bge-base-en-v1.5"
os.environ["RERANK_MODEL_ID"] = "BAAI/bge-reranker-base"
os.environ["LLM_MODEL_ID"] = "Intel/neural-chat-7b-v3-3"

## 2.2 Start the containers

All containers are already created and you can pick your components for your example based on your needs. This is done through a docker compose file, for this example, we will be using the file available on the repo to use `Milvus` as Vector Data base. You can explore some `docker compose` files provisioned for ChatQnA [HERE](link), or you can build your own to use the multiple available components on [GenAIComps](repo)

In [9]:
# Cell output was intentionally hidden.
%%capture captured_output

!cd GenAIExamples/ChatQnA/docker_compose/intel/cpu/xeon/ && docker compose -f compose_milvus.yaml up -d

## 2.3 Verify Containers

In [4]:
# You should be able to see 8 containers
!docker ps

CONTAINER ID   IMAGE                                                   COMMAND                  CREATED         STATUS                            PORTS                                                                                  NAMES
71b0592c776d   opea/nginx:latest                                       "/docker-entrypoint.…"   3 minutes ago   Up 3 minutes                      0.0.0.0:80->80/tcp, :::80->80/tcp                                                      chatqna-xeon-nginx-server
ebf6be0c36eb   opea/chatqna-ui:latest                                  "docker-entrypoint.s…"   3 minutes ago   Up 3 minutes                      0.0.0.0:5173->5173/tcp, :::5173->5173/tcp                                              chatqna-xeon-ui-server
edf2c1bbc933   opea/chatqna:latest                                     "python chatqna.py"      3 minutes ago   Up 3 minutes                      0.0.0.0:8888->8888/tcp, :::8888->8888/tcp                                              chatqna-xeon-

In [5]:
#Check vLLM - Wait Until you see vLLM started complete

!docker logs vllm-service 2>&1 | grep complete

INFO 02-10 15:06:36 __init__.py:190] Automatically detected platform cpu.
WARNING 02-10 15:06:37 _logger.py:72] Torch Profiler is enabled in the API server. This should ONLY be used for local development!
INFO 02-10 15:06:37 api_server.py:840] vLLM API server version 0.1.dev1+gfde7126
INFO 02-10 15:06:37 api_server.py:841] args: Namespace(host='0.0.0.0', port=80, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, chat_template_content_format='auto', response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_request_id_headers=False, enable_auto_tool_choice=False, enable_reasoning=False, reasoning_parser=None, tool_call_parser=None, tool_parser_plugin='', model='Intel/neural-chat-7b-v3-3', task=

# 3. Test your application

ChatQnA is an end-to-end blueprint, which means that in addition to the components mentioned before, you will also have an UI to interact with your application.

## 3.1 Use the UI 

To test on your browser:

1. Copy your Jupyter LAB URL, for example:

`https://tiber-opea-workshop-1.eglb.intel.com/2/lab/tree/opea.ipynb`

2. Get your public token for the URL. 

In [ ]:
print(os.environ["PUBLICURL"])

3. Build you new URL, adding the PUBLICURL value at the end and using the same format

`https://tiber-opea-workshop-1.eglb.intel.com/opea/2/95ca4f808974c40bead0d6eebe10714a/`

4. You should be able to see the public UI
![IMAGE](./Images/opea_ui.png)

You can now interact with the application, we can ask things like **"What is Nike's revenue in 2023?"**.

Let's now explore each microservice and their behaviour, this will help you understand how they interact and how the RAG example is built

## 3.2 Prompt from the megaservice

In [ ]:
def query_chatqna(query: str, verbose: bool = True) -> str:
    # Define the API endpoint and headers
    url = "http://localhost/v1/chatqna"
    headers = {"Content-Type": "application/json"}
    payload = {"messages": "What is the revenue of Nike in 2023?"}

    # Make the request and stream the response
    content = ""
    with requests.post(url, json=payload, headers=headers, stream=True) as response:
        for line in response.iter_lines():
            if line:  # Ignore empty lines
                decoded_line = line.decode("utf-8")
                if decoded_line == "[DONE]":
                    break  # Stop when the stream is done
                decoded_clean = (
                    decoded_line.replace("data: b'", "")
                    .replace("'", "")
                    .replace("data: [DONE]", "")
                )
                content += decoded_clean
                if verbose:
                    print(decoded_clean, end="", flush=True)

    return content

You should receive an answer back, verifying that all the services in the RAG flow are working, remember that the microservice is the point of entry and interacts with all the internal services, if you recieve an answer will mean that the blueprint is working.

In [ ]:
# Query ChatQnA without RAG
query = "What was the revenue of Nike in 2023?"
content = query_chatqna(query)


You will see that the model couldn’t answer the question because it lacks the necessary context and operates on ***outdated information***. Without up-to-date or relevant data, it’s unable to provide accurate responses. You'll fix this in few minutes by using RAG to allow the model to pull in current, contextually relevant information, ensuring more precise and relevant answers.

## 3.3 Provide external context before prompting 

In this step, you will be feeding the application with context information. From the previous query, the model didn't know **"What is the revenue of Nike in 2023?"**, affortunately, if we have a document containing that information we could feed our application and have it correclty answered. The process then,  is to start from a document (Nike's revenue PDF), and do the preprocessing needed to make it ready to be stored in a database. As shown, this process primarily involves 3 microservices: `data preparation`, `embeddings` and `vector store`. Let's explore each microservice
<p align="center">
  <img src="../Images/pre_flow.png" alt="Alt text" />
</p>

### 3.3.1 Embedding Microservice (Container:chatqna-tei:80)

The document has to be ingested in a way that operations like similarities and storage can be done. Documents aren't stored with words, they are stored as `embeddings`.
An **embedding** is a numerical representation of an object—such as a word, phrase, or document in a continuous vector space. In the context of natural language processing (NLP), embeddings represent words, sentences, or other pieces of text as a set of numbers (or a "vector") that capture their meaning, relationships, and context. By transforming text into this format, embeddings make it easier for machine learning models to understand and work with text data.

For example, the following image shows how word embeddings represent words as points in a vector space based on their relationships. Words with similar meanings, like "king" and "queen" are closer together, and the embedding model captures these connections through vector arithmetic. 

During training, if the model sees "king" often used with "man" and "queen" with "woman," it learns that "king" and "queen" relate similarly to "man" and "woman." So, it positions these words in ways that reflect gender relationships in language.

<p align="center">
  <img src="./Images/king_vs_queen.png" alt="Alt text" />
</p>


Embeddings are a key component for RAG:

•	***Capturing Meaning***: Embeddings represent the semantic relationships between words, allowing RAG models to understand context and nuances in language, enhancing their ability to generate relevant responses.

•	***Dimensionality Reduction***: By converting complex information into fixed-size vectors, embeddings streamline data processing, making RAG systems more efficient and faster.

•	***Improving Model Performance***: Embeddings enable RAG models to generalize better by leveraging semantic similarities, facilitating more accurate information retrieval, and improving the quality of generated content.

OPEA provides multiple options to run your embeddings microservice, as detailed in the [OPEA embedding documentation](https://github.com/opea-project/GenAIComps/tree/main/comps/embeddings): 

In this case, ChatQnA uses [Hugging Face TEI](https://huggingface.co/docs/text-embeddings-inference/en/index) microservice running the embedding model `BAAI/bge-large-en-v1.5` locally. 

Test the microservice by getting the embedding for the phrase "What was Deep Learning?":

In [ ]:
!curl http://localhost:6006/embed -X POST -d \
    '{"inputs":"What is Deep Learning?"}' -H \
    'Content-Type: application/json'

### 3.3.2. Store the public Nike report for 2023 in the Vector Database (chatqna-data-prep:6007)

As mentioned, this example uses Milvus as the vector database. You can find all of the supported alternatives in the [OPEA vector store repository](https://github.com/opea-project/GenAIComps/tree/main/comps/vectorstores)

A Vector Database (VDB) is a specialized database designed to store and manage high-dimensional vectors—numeric representations of data points like words, sentences, or images. In AI and machine learning, these vectors are typically embeddings, which capture the meaning and relationships of data in a format that algorithms can process efficiently, as we have shown before.

The Microservice involved on storing the emebddings to the vdb is the `Dataprep` Microservice. It is responsible for preparing data in a digestible format for the application, converting it to embeddings (using the embedding microservice) and loading it to the database. This service preprocesses/transforms the data, making sure it is clean, organized, and suitable for further processing.  

Specifically, this microservice receives data (such as documents), processes it by breaking it into chunks, sends it to the embedding microservice, and stores these vectors in the vector database. The microservice's functionality may depend on the specific vector database being used, as each database has its own requirements for data formatting

To test it and help the model answer the initial question **What was Nike revenue in 2023?**, you will need to upload a context file (revenue report) to be processed. 

1. Execute the following command to download a sample [Nike revenue report](https://github.com/opea-project/GenAIComps/blob/main/comps/retrievers/redis/data/nke-10k-2023.pdf) to the nginx pod (if you are no longer logged in to the NGinx pod, be sure to use the above command to log in again):

In [ ]:
# Ingest your data for your RAG. In this case NIke's revenue document

!wget https://raw.githubusercontent.com/opea-project/GenAIComps/v1.1/comps/retrievers/redis/data/nke-10k-2023.pdf

2. Run the ingestion function on the `dataprep` microservice.

In [ ]:
!curl -X POST "http://localhost:11101/v1/dataprep/ingest" \
     -H "Content-Type: multipart/form-data" \
     -F "files=@./nke-10k-2023.pdf"

3. Run the get function to retrieve all documents ingested on the vdb

In [ ]:
# You can check if the file was successfully updated
!curl -X POST "http://localhost:11101/v1/dataprep/get" \
     -H "Content-Type: application/json"

## 3.4 Test if your RAG application provide answer based on context (Nike revenue in 2023)

1. Run your query (Directly to the LLM)

In [8]:
# You can directly check the LLM,
!curl http://${host_ip}:9009/v1/chat/completions \
  -X POST \
  -d '{"model": "Intel/neural-chat-7b-v3-3", "messages": [{"role": "user", "content": "What is Deep Learning?"}], "max_tokens":17}' \
  -H 'Content-Type: application/json'

^C


You can verify that the model doesn't answered correctly, but WHY? 

The answer is simple, the LLM isn't re trained with the context, the information you added in the previous step (Nike revenue) is on the vector database. This information is given as a context (new prompt) to the LLM when is prompted from the entry point `megaservice` which will start the process of recieving the answer, finding the most similar documents (chunks) on the vector database using the `retriever` microservice and finally creating a new prompt that is ingested to the LLM.

2. Query ChatQnA WITH RAG

In [ ]:
query = "What is the revenue of Nike in 2023?"
content = query_chatqna(query)

# Be involved with OPEA

OPEA is an open source project that welcomes contributions in many forms. Whether you're interested in fixing bugs, adding new GenAI components, improving documentation, or sharing your unique use cases, there are numerous ways to get involved and make a meaningful impact. 

We’re excited to have you on board and look forward to the contributions you'll bring to the OPEA platform. 

For detailed instructions on how to contribute, please check out our [Contributing Documentation](https://opea-project.github.io/latest/community/CONTRIBUTING.html).

[Follow the project](https://github.com/opea-project) and stay tuned for new events!
